In [20]:
import numpy as np
import pylab as plt
from scipy.signal import resample,correlate2d
from scipy.ndimage.filters import gaussian_filter

def fft(A,n=None):
    '''exp{-i2pi (lu + mv)}'''
    return np.fft.fftshift(np.fft.fftn(np.fft.ifftshift(A),s=n))

def ifft(A,n=None):
    '''exp{i2pi (lu + mv)}'''
    return np.fft.fftshift(np.fft.ifftn(np.fft.ifftshift(A),s=n))

def conv2d(A,B):
    if np.size(A) != np.size(B):
        print("wrong sizes")
        return
    return np.fft.ifft2(np.fft.fft2(A)*np.fft.fft2(B))

def autocorr(A):
    F = fft(A)
    return ifft(F*np.conjugate(F))

def regrid(A,shape,*presampledAx):
    '''Uses fft to regrid ...'''
    n = len(shape)
    if len(presampledAx) != n:
        print("wrongsize sample axis")
        return
    B = np.copy(A)
    resampledAx = []
    i = 0
    while i < n:
        B,t = resample(B,shape[i],t=presampledAx[i],axis=i)
        resampledAx.append(t)
        i += 1
    return B,resampledAx

def dft2(A,L,M,x,y):
    res = np.zeros_like(L,dtype=type(1j))*1j

    n = 0
    while n < np.size(x):
        print(n/np.size(x))
        p = 0
        while p < np.size(y):
            res += A[n,p]*np.exp(-1j*2.*np.pi*(x[n]*L + y[p]*M))
            #print(res[i,j])
            p += 1
        n += 1
       
           
    return res

def complexGaussianFilter(A,sigma=3,order=0):
    return gaussian_filter(np.real(A),sigma=sigma,order=order) + 1j*gaussian_filter(np.imag(A),sigma=sigma,order=order)

'''propagate distortions'''
l = np.linspace(-0.5,0.5,100)

A = np.exp(-1j*2*np.pi*np.sqrt(1-l**2))
U = ifft(A)
#plt.plot(U)
#plt.show()
#plt.plot(fft(U))
#plt.show()
A = np.random.uniform(size=[5,5,5])
x = np.linspace(0,1,5)
B,y = regrid(A,[10,11,120],*(x,x,x))
print (B)
print (y)
#plt.imshow(B[0,:,:],interpolation='nearest')
#plt.show()
w = 100000
up = np.linspace(-10,10,1000)
dx=np.abs(up[1]-up[0])
U,V = np.meshgrid(up,up)
l = np.fft.fftshift(np.fft.fftfreq(1000,d=dx))
dl = l[1]-l[0]
m = l
L,M = np.meshgrid(l,m)
u = 0
v = 0
#k = np.exp(1j*2*np.pi*w)*np.exp(-1j*np.pi*w*(U**2+V**2))
k = complexGaussianFilter(np.exp(1j*2*np.pi*w*(-2/w**2*(u*U + v*V) + V**2/w**2 + U**2/w**2)))
#k = w/(1j*distance)*exp(1j*2*np.pi*0.5/distance*Z)
#Af = dft2(k,L,M,up,up)*dx**2

Af = fft(k)*dx**2
A = complexGaussianFilter(1j*w/2.*np.exp(-1j*np.pi/2./w*(4*L*u*w + 4*M*v*w + 4*(u**2+v**2) + (L**2 + M**2)*w**2)))
kf = ifft(A/dx**2)
import pylab as plt
#print(np.mean(np.abs(kf)))
plt.imshow(np.angle (Af),interpolation='nearest',origin='lower')
plt.colorbar()
plt.show()
    

[[[ 0.90544314  0.94113566  0.97317941 ...,  0.78049654  0.82469102
    0.86648865]
  [ 0.91441609  0.94664793  0.97492082 ...,  0.79742731  0.83942686
    0.87855452]
  [ 0.69998748  0.70578472  0.70851829 ...,  0.6647575   0.67938966
    0.69116597]
  ..., 
  [ 0.09527776  0.08603157  0.07844012 ...,  0.13251008  0.11857186
    0.10614095]
  [ 0.24596403  0.23984089  0.23357598 ...,  0.26354587  0.25780198
    0.25194755]
  [ 0.61044662  0.62588771  0.63915057 ...,  0.55322377  0.57387514
    0.59303275]]

 [[ 0.50438698  0.52330388  0.54165564 ...,  0.4465701   0.46578537
    0.48513601]
  [ 0.82901879  0.8410619   0.84997328 ...,  0.77575953  0.79617416
    0.81399609]
  [ 0.97789455  0.96809416  0.95328746 ...,  0.97653558  0.98213547
    0.98259044]
  ..., 
  [ 0.34106399  0.32827853  0.3134806  ...,  0.36591613  0.35999987
    0.35168125]
  [ 0.24813947  0.23814942  0.22685694 ...,  0.26925657  0.26377545
    0.25671262]
  [ 0.2668733   0.27319975  0.27954184 ...,  0.24882796  0

In [6]:
help(plt.imshow
)


Help on function imshow in module matplotlib.pyplot:

imshow(X, cmap=None, norm=None, aspect=None, interpolation=None, alpha=None, vmin=None, vmax=None, origin=None, extent=None, shape=None, filternorm=1, filterrad=4.0, imlim=None, resample=None, url=None, hold=None, data=None, **kwargs)
    Display an image on the axes.
    
    Parameters
    -----------
    X : array_like, shape (n, m) or (n, m, 3) or (n, m, 4)
        Display the image in `X` to current axes.  `X` may be a float
        array, a uint8 array or a PIL image. If `X` is an array, it
        can have the following shapes:
    
        - MxN -- luminance (grayscale, float array only)
        - MxNx3 -- RGB (float or uint8 array)
        - MxNx4 -- RGBA (float or uint8 array)
    
        The value for each component of MxNx3 and MxNx4 float arrays
        should be in the range 0.0 to 1.0; MxN float arrays may be
        normalised.
    
    cmap : `~matplotlib.colors.Colormap`, optional, default: None
        If None, d

In [116]:
x = np.linspace(0,1,5)
X,Y = np.meshgrid(x,x)
l = np.fft.fftfreq(5,x[1]-x[0])
print (l)

[ 0.   0.8  1.6 -1.6 -0.8]
